In [125]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm 
import h5py
import cv2

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
from segment_anything.utils.transforms import ResizeLongestSide

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
DTYPE  = torch.float32

In [2]:
# First, we load and preprocess the data

data_path = "/Volumes/ES-HDD-Documents/Documents/CFHT_galaxies_with_streams/"
label_path = "/Volumes/ES-HDD-Documents/Documents/CFHT_galaxies_with_streams/streams_masks/"
data_list = np.loadtxt(data_path + "list_asinh.txt", dtype=str)
label_list = np.loadtxt(label_path + "/list_mask.txt", dtype=str)

In [127]:
with h5py.File(data_path+'galaxy_stream.h5', 'r') as f:
    images = f['images'][:]
    masks  = f['masks'][:]

In [128]:
N = len(images)
bbox_coords = np.zeros([N, 4]) + np.array([0,0,3333,3333])

In [130]:
sam_checkpoint = "/Users/davidchemaly/Weights/sam_vit_h_4b8939.pth"
model_type = "vit_h"

sam_model = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam_model.to(device=DEVICE)
sam_model.train();

In [97]:
class MyDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = {'data': self.data[idx], 'label': self.labels[idx]}
        return sample
    
class MyDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = {'data': self.data[idx], 'label': self.labels[idx]}
        return sample

    def __str__(self):
        return f"Dataset({{\n    features: ['image', 'label'],\n    num_rows: {len(self)}\n}})"

    
dataset = MyDataset(images, masks)

In [122]:
def get_bounding_box(ground_truth_map):
  # # get bounding box from mask
  # _, y_indices, x_indices = np.where(ground_truth_map > 0)
  # x_min, x_max = np.min(x_indices), np.max(x_indices)
  # y_min, y_max = np.min(y_indices), np.max(y_indices)
  # # add perturbation to bounding box coordinates
  # C, H, W = ground_truth_map.shape
  x_min = 0 #max(0, x_min - np.random.randint(0, 20))
  x_max = ground_truth_map.shape[-2] #min(W, x_max + np.random.randint(0, 20))
  y_min = 0 #max(0, y_min - np.random.randint(0, 20))
  y_max = ground_truth_map.shape[-1] #min(H, y_max + np.random.randint(0, 20))
  bbox = [x_min, y_min, x_max, y_max]

  return bbox

class SAMDataset(Dataset):
  def __init__(self, dataset, processor):
    self.dataset = dataset
    self.processor = processor

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    item = self.dataset[idx]
    image = item["data"]
    ground_truth_mask = np.array(item["label"])

    # get bounding box prompt
    prompt = get_bounding_box(ground_truth_mask)

    # prepare image and prompt for the model
    inputs = self.processor(image)#, input_boxes=[[prompt]], return_tensors="pt")

    # remove batch dimension which the processor adds by default
    inputs = {k:v.squeeze(0) for k,v in inputs.items()}

    # add ground truth segmentation
    inputs["ground_truth_mask"] = ground_truth_mask

    return inputs

In [62]:
# Load SAM Model for Fine Tunning

sam_checkpoint = "/Users/davidchemaly/Weights/sam_vit_h_4b8939.pth"
model_type = "vit_h"

sam_model = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam_model.to(device=DEVICE);

In [94]:
from transformers import SamProcessor

processor = SamProcessor.from_pretrained("facebook/sam-vit-base")

In [123]:
image_embedding = sam_model.image_encoder
train_dataset = SAMDataset(dataset=dataset, processor=image_embedding)

In [124]:
for i in train_dataset:
    print(i)
    break

TypeError: conv2d() received an invalid combination of arguments - got (numpy.ndarray, Parameter, Parameter, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, tuple of ints padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!numpy.ndarray!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, int)
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, str padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!numpy.ndarray!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, int)
